In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os, time, random
import random
import cv2

In [3]:
train_data = np.load(r"train_data.npy")
val_data = np.load(r"val_data.npy")
test_data = np.load(r"test_data.npy")

In [4]:
train_data.shape

(100000, 64, 64, 3)

### Create patches and preprocess them

Our images are of size 64x64, so we have to choose an appropriate size for the patch. We maintain pretty much the same number of patches within a single images (roughly around 32 in the original paper) by choosing 12x12 patches. We allow for a gap of around half the size of the patch and we jitter the location of the patch by -2 to 2 pixels in each direction.

In [5]:
def sample_patch(x, y, grid_start_x, grid_start_y, patch_size, gap_size, jitter_size, image_size):

    #initial position of the patch
    x_start = grid_start_x + x * (patch_size[1] + gap_size) + random.randint(-jitter_size, jitter_size)
    y_start = grid_start_y + y * (patch_size[0] + gap_size) + random.randint(-jitter_size, jitter_size)

    #ensure the patch stays inside image boundaries
    x_start_2 = min(max(x_start, 0), image_size[1] - patch_size[1])
    y_start_2 = min(max(y_start, 0), image_size[0] - patch_size[0])

    #return the coordinates of the upper-left corner
    return (x_start_2, y_start_2)

In [1]:
def preprocess_patch(patch):
    #randomly drop all but one color channel
    kept_channel = random.randint(0, 2)

    mean = [123.68, 116.779, 103.939]

    for i in range(0, 3):
        if i == kept_channel:
            patch[:,:,i] -= np.mean(patch[:,:,i])
        else:
            patch[:,:,i] = np.random.uniform(0, 1, (patch.shape[0], patch.shape[1])) - 0.5

    #normalize mean and variance
    patch = patch/np.sqrt(np.mean(patch**2))*50

    return patch